In [ ]:
import librosa
import librosa.display
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import seaborn as sns
import sys
import pandas as pd

sys.path.append("../utils")
from bitcode import *
from utils import *
from AlignedDvSound import *

%load_ext autoreload
%autoreload 2

In [ ]:
out_dir = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\out"

aedat_dir = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\aedat"
aedat_fname = 'dvSave-2022_04_02_03_18_43.aedat4'
dv_path = os.path.join(aedat_dir, aedat_fname)

wav_dir = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\wav"
wav_name = '20220401_2_main.wav'
wv_path = os.path.join(wav_dir, wav_name)

In [ ]:
# choose align roi
with AedatFile(dv_path) as f:
    # for frame in f['frames']:
    img = next(f['frames']).image

align_roi = cv2.selectROI(img)
cv2.destroyAllWindows()

# wing_roi = cv2.selectROI(img)
# cv2.destroyAllWindows()
print(align_roi)

In [ ]:
aligned = AlignedDvSound(dv_path, wv_path)
# aligned.alignFromCodes(dv_codes=dv_code, wv_codes=wv_code)
aligned.autoAlign(time_window=[30, 50], roi=align_roi)

In [ ]:
print(aligned.align)

In [ ]:
# choose align roi
with AedatFile(dv_path) as f:
    # for frame in f['frames']:
    img = next(f['frames']).image

# align_roi = cv2.selectROI(img)
# cv2.destroyAllWindows()

wing_roi = cv2.selectROI(img)
cv2.destroyAllWindows()

In [ ]:
print(wing_roi)

In [ ]:
slice = aligned.getSlice([7*60+30,7*60+35], coord='wv', roi=wing_roi)
plt.plot(np.arange(slice['dv']['x'].shape[0]) / slice['dv']['sr'], normalize(slice['dv']['x']), linewidth=0.5)
plt.plot(np.arange(slice['wv']['x'].shape[0]) / slice['wv']['sr'], normalize(slice['wv']['x']), linewidth=0.5)

In [ ]:
import scipy

plt.figure(figsize=[20,5], dpi=300)

b, a = scipy.signal.butter(3, Wn=2*200/slice['wv']['sr'], btype='highpass', analog=False, output='ba')
x = scipy.signal.filtfilt(b, a, slice['wv']['x'])
plt.plot(np.arange(x.shape[0]) / slice['wv']['sr'], normalize(x), linewidth=0.5)

b, a = scipy.signal.butter(3, Wn=2*200/slice['dv']['sr'], btype='highpass', analog=False, output='ba')
x = scipy.signal.filtfilt(b, a, slice['dv']['x'])
plt.plot(np.arange(x.shape[0]) / slice['dv']['sr'], normalize(x) + 20, linewidth=0.5)

In [ ]:
slice['wv']['sr']

In [ ]:
show_spec(slice['wv']['x'], slice['wv']['sr'])